In [924]:
import pandas as pd
import matplotlib.pyplot as plt

In [925]:

def renameColumns(df):
    return [col.split('-', 1)[1] if '-' in col else col for col in df.columns]

def convertTime(df):
    if df["Time"].dtype == 'int64':
        df["Time"] = pd.to_datetime(df["Time"], unit='ms')    
    else:
        df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')
    df.sort_values(by=['Time'], inplace=True)
    return df

In [926]:
def readAndProcessCSV(filename):
    df = pd.read_csv(filename)
    df.columns = renameColumns(df)
    df = convertTime(df)
    return df

In [927]:
def isWeekend(dateObj):
    return dateObj.weekday() in [5, 6]

def isWeeekday(dateObj):
    return not isWeekend(dateObj)

#Depending on on the mask created, you can get the data from the weekend or the week
def applyMask(df, mask):
    dfWeekend = df[~mask]
    dfWeekend = dfWeekend.reset_index(drop=True)
    return dfWeekend

def eliminateNightHours(df):
    return df[(df['Time'].dt.hour >= 6) & (df['Time'].dt.hour <= 23)]

def getDataOneHour(hour, df):
    return df[(df['Time'].dt.hour > hour) & (df['Time'].dt.hour <= hour + 1)]

def getDataOneDay(date, df):
    return df[(df['Time'].dt.day == date.day) & (df['Time'].dt.month == date.month) & (df['Time'].dt.year == date.year)]

def getDataThreeDays(date, df):
    return df[(df['Time'].dt.day >= date.day) & (df['Time'].dt.day <= date.day + 2) & (df['Time'].dt.month == date.month) & (df['Time'].dt.year == date.year)]

def eliminateDayHours(df):
    return df[(df['Time'].dt.hour > 6) & (df['Time'].dt.hour < 23)]

In [928]:
def makeDfSensorSingle(df, sensorName):
    sensorColumns = ['Time'] + [col for col in df.columns if sensorName in col] 
    dfSensor = df[sensorColumns]
    dfSensor = dfSensor.dropna(axis=1, how='all')
    dfSensor = dfSensor.dropna().reset_index(drop=True)
    return dfSensor

In [929]:
#DF that has all the data from October and November
dfOctNov = readAndProcessCSV("sensorData/08_45_18.csv")
fileNames = ["08_46_27", "08_47_03", "08_48_14", "08_48_51", "08_49_32", "08_50_13", "08_50_46"]
for fileName in fileNames:
    df = readAndProcessCSV("sensorData/" + fileName + ".csv")
    dfOctNov = pd.concat([dfOctNov, df])

#Time range of dfOctNov is 2023-10-01 to 2023-11-25
#dfOctNov["Time"].dt.date

In [930]:
#DF that has only date time data (6 AM to 11 PM)
dfDay = eliminateNightHours(dfOctNov)
dfDay = dfDay.reset_index(drop=True)

In [931]:

#dfWeekend - df that contains only the data from the weekend
weekendMask = dfOctNov['Time'].apply(isWeekend)
dfWeekend = applyMask(dfOctNov, weekendMask)

In [932]:
#dfWeekday - df that contains only the data from the week
weekdayMask = dfOctNov['Time'].apply(isWeeekday)
dfWeekday = applyMask(dfOctNov, weekdayMask)

In [933]:
dfFinal = applyMask(dfOctNov, weekendMask)
dfFinal = eliminateNightHours(dfFinal)
dfFinal = dfFinal.reset_index(drop=True)

In [934]:
def getTemperatureMean(df, sensorName, measurementName):
    columnName = sensorName + '.' + measurementName
    dfCopy = getDataThreeDays(pd.to_datetime('2023-10-16'), df)
    return dfCopy[columnName].mean()

def getTemperatureStd(df, sensorName, measurementName):
    columnName = sensorName + '.' + measurementName
    dfCopy = getDataThreeDays(pd.to_datetime('2023-10-16'), df)
    return dfCopy[columnName].std()

temperatureMean = {}
for i in range(1, 10):
    sensorName = 'eye0' + str(i)
    temperatureMean.update({sensorName: getTemperatureMean(dfFinal, sensorName, 'temperature')})
temperatureMean.update({'eye11': getTemperatureMean(dfFinal, 'eye11', 'temperature')})
temperatureMean.update({'eye10': getTemperatureMean(dfFinal, 'eye10', 'temperature')})

for i in range(1, 7):
    sensorName = 'gas0' + str(i)
    temperatureMean.update({sensorName: getTemperatureMean(dfFinal, sensorName, 'temperature')})

for i in range(1, 6):
    sensorName = 'voc0' + str(i)
    temperatureMean.update({sensorName: getTemperatureMean(dfFinal, sensorName, 'temperature')})

#sort the dictionary by value
temperatureMean = dict(sorted(temperatureMean.items(), key=lambda item: item[1]))

for sensorName in temperatureMean:  
    print(sensorName, temperatureMean[sensorName])  

eye07 20.780128205128207
voc04 20.986068111455136
voc01 21.323725834797884
gas02 21.863975155279515
gas05 22.367973856209133
gas03 22.434890965732098
gas04 22.457763975155277
eye09 22.523841059602677
gas06 22.560691823899358
voc05 22.56228748068004
gas01 22.709375
eye11 22.713522012578615
voc02 22.797927461139917
voc03 22.845679012345677
eye02 22.992569659442708
eye06 23.11246105919004
eye08 23.216780821917812
eye05 23.58699690402475
eye01 23.623701298701306
eye04 23.991536050156753
eye10 24.538699690402492
eye03 25.227244582043344


In [935]:
temperatureStd = {}
for i in range(1, 10):
    sensorName = 'eye0' + str(i)
    temperatureStd.update({sensorName: getTemperatureStd(dfFinal, sensorName, 'temperature')})
temperatureStd.update({'eye11': getTemperatureStd(dfFinal, 'eye11', 'temperature')})
temperatureStd.update({'eye10': getTemperatureStd(dfFinal, 'eye10', 'temperature')})

for i in range(1, 7):
    sensorName = 'gas0' + str(i)
    temperatureStd.update({sensorName: getTemperatureStd(dfFinal, sensorName, 'temperature')})

for i in range(1, 6):
    sensorName = 'voc0' + str(i)
    temperatureStd.update({sensorName: getTemperatureStd(dfFinal, sensorName, 'temperature')})
temperatureStd = dict(sorted(temperatureStd.items(), key=lambda x: x[1]))

for sensorName in temperatureStd:  
    print(sensorName, temperatureStd[sensorName])  

voc01 0.5174587505337246
gas02 0.5690405977371287
gas05 0.5796724450726946
gas04 0.6018807233812346
eye04 0.6053928173664935
eye02 0.6237090199452495
gas03 0.674789831132166
eye11 0.6935754193236257
voc04 0.7496713626268409
eye03 0.773353284953186
voc03 0.7882521321605113
gas01 0.80354207041957
voc02 0.8639227275387978
voc05 0.9164305288078032
eye09 0.9361125957542558
gas06 0.945483448848527
eye05 0.9698968809196752
eye07 1.0212482036598352
eye06 1.024344540064584
eye10 1.1375115232096273
eye08 1.2781104152548117
eye01 1.5121839379779294


eye10 31.74613003095975
voc02 31.98073555166375
eye02 34.298136645962735
gas05 34.642857142857146
voc01 36.06818181818182
voc03 37.672360248447205
gas01 38.46875
voc05 41.040247678018574
voc04 41.244204018547144
gas04 43.250773993808046
gas03 43.50931677018634
gas06 44.043887147335425
gas02 47.75077881619938
--------------------------
voc04 20.986068111455136
voc01 21.323725834797884
gas02 21.863975155279515
gas05 22.367973856209133
gas03 22.434890965732098
gas04 22.457763975155277
gas06 22.560691823899358
voc05 22.56228748068004
gas01 22.709375
voc02 22.797927461139917
voc03 22.845679012345677
eye02 22.992569659442708
eye10 24.538699690402492


0         2023-10-02
1         2023-10-02
2         2023-10-02
3         2023-10-02
4         2023-10-02
             ...    
120268    2023-11-24
120269    2023-11-24
120270    2023-11-24
120271    2023-11-24
120272    2023-11-24
Name: Time, Length: 120273, dtype: object

In [938]:
#Sensor reading for eye01 - every 10 minutes 00:09:38
#Sensor reading for eye02 - every 10 minutes 00:07:37 
#Sensor reading for eye03 - every 10 minutes 00:06:08

sensorEye01 = makeDfSensorSingle(dfDay, 'eye02')
sensorGas01 = makeDfSensorSingle(dfOctNov, 'gas01')
sensorVOC01 = makeDfSensorSingle(dfOctNov, 'voc01')

dtype('O')

In [939]:
#sensorEye01 = sensorEye01[sensorEye01['eye01.occupancy'] == 1]
sensorEye01.head()

,Time,eye02.humidity,eye02.light,eye02.motion,eye02.occupancy,eye02.temperature,eye02.vdd
0,2023-10-01 06:07:36,36.0,1.0,0.0,0.0,23.4,3659.0
1,2023-10-01 06:17:36,36.0,1.0,0.0,0.0,23.4,3659.0
2,2023-10-01 06:27:36,36.0,1.0,0.0,0.0,23.4,3659.0
3,2023-10-01 06:37:36,36.0,1.0,0.0,0.0,23.3,3661.0
4,2023-10-01 06:47:36,36.0,1.0,0.0,0.0,23.4,3659.0


In [940]:
sensorVOC01.head()

,Time,voc01.humidity,voc01.light,voc01.motion,voc01.temperature,voc01.tvoc,voc01.vdd
0,2023-10-01 00:00:23,30.0,0.0,0.0,25.3,131.0,3648.0
1,2023-10-01 00:05:23,30.0,0.0,0.0,25.3,119.0,3648.0
2,2023-10-01 00:10:23,31.0,0.0,0.0,25.3,135.0,3648.0
3,2023-10-01 00:15:23,31.0,0.0,0.0,25.3,128.0,3648.0
4,2023-10-01 00:20:23,31.0,0.0,0.0,25.3,132.0,3648.0


In [941]:
sensorGas01.head()

,Time,gas01.co2,gas01.humidity,gas01.light,gas01.motion,gas01.temperature,gas01.vdd
0,2023-10-01 06:05:59,447.0,35.0,0.0,0.0,25.6,3667.0
1,2023-10-01 06:15:59,437.0,35.0,0.0,0.0,25.6,3664.0
2,2023-10-01 06:25:59,440.0,35.0,0.0,0.0,25.6,3667.0
3,2023-10-01 06:35:59,429.0,35.0,0.0,0.0,25.7,3667.0
4,2023-10-01 06:45:58,434.0,35.0,0.0,0.0,25.6,3670.0
